In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
# Map StatisticGroup into broader categories
def categorize_statistic_group(stat_group):
    categories = {
        "עבירות פליליות כלליות": ['עבירות כלפי הרכוש', 'עבירות נגד גוף', 'עבירות מין'],
        "עבירות מוסר": ['עבירות מוסר', 'עבירות סדר ציבורי'],
        "עבירות ביטחון": ['עבירות בטחון'],
        "עבירות כלכליות": ['עבירות כלכליות', 'עבירות מנהליות', 'עבירות רשוי'],
        "עבירות תנועה": ['עבירות תנועה'],
        "עבירות מרמה": ['עבירות מרמה'],
    }
    for category, types in categories.items():
        if stat_group in types:
            return category
    return None # Default for uncategorized types


In [3]:
def preprocess_data(df):
    """
    מבצעת ניקוי וטרנספורמציה על הדאטה:
    1. מסירה רשומות עבור מחוזות "כל הארץ" ו-"" (ריק).
    2. יוצרת מחוז חדש בשם "כל המחוזות" המכיל את סך כל הנתונים של שאר המחוזות.
    """
    # שלב 1: סינון רשומות לא רצויות
    filtered_df = df[~df["PoliceDistrict"].isin(["כל הארץ", ""])]

    # שלב 2: יצירת מחוז חדש "כל המחוזות"
    aggregated_df = filtered_df.groupby(["Category", "Period"]).agg({"Count": "sum"}).reset_index()
    aggregated_df["PoliceDistrict"] = "כל המחוזות"

    # שלב 3: איחוד הנתונים חזרה
    combined_df = pd.concat([filtered_df, aggregated_df], ignore_index=True)

    return combined_df


In [4]:
# קריאה לפונקציה
#processed_df = preprocess_data(grouped)

# סינון ועיבוד הדאטה
# grouped = df.groupby(["Category", "Period", "PoliceDistrict"]).size().reset_index(name="Count")
# grouped = preprocess_data(grouped)


NameError: name 'df' is not defined

In [7]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import requests

# Function to load and process data
def load_data():
    urls = {
        "2020": "https://data.gov.il/api/3/action/datastore_search?resource_id=520597e3-6003-4247-9634-0ae85434b971",
        "2021": "https://data.gov.il/api/3/action/datastore_search?resource_id=3f71fd16-25b8-4cfe-8661-e6199db3eb12",
        "2022": "https://data.gov.il/api/3/action/datastore_search?resource_id=a59f3e9e-a7fe-4375-97d0-76cea68382c1",
        "2023": "https://data.gov.il/api/3/action/datastore_search?resource_id=32aacfc9-3524-4fba-a282-3af052380244",
        "2024": "https://data.gov.il/api/3/action/datastore_search?resource_id=5fc13c50-b6f3-4712-b831-a75e0f91a17e",
    }
    data_frames = []
    for year, url in urls.items():
        response = requests.get(url)
        data = response.json()
        records = data['result']['records']
        df = pd.DataFrame(records)
        df['Year'] = int(year)  # Add year column
        df = df.dropna(subset=["StatisticGroup"])  # Remove rows with missing StatisticGroup
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

# Map StatisticGroup into broader categories
def categorize_statistic_group(stat_group):
    categories = {
        "עבירות פליליות כלליות": ['עבירות כלפי הרכוש', 'עבירות נגד גוף', 'עבירות מין'],
        "עבירות מוסר": ['עבירות מוסר', 'עבירות סדר ציבורי'],
        "עבירות ביטחון": ['עבירות בטחון'],
        "עבירות כלכליות": ['עבירות כלכליות', 'עבירות מנהליות', 'עבירות רשוי'],
        "עבירות תנועה": ['עבירות תנועה'],
        "עבירות מרמה": ['עבירות מרמה'],
    }
    for category, types in categories.items():
        if stat_group in types:
            return category
    return None  # Return None for uncategorized values

# Preprocessing function
def preprocess_data(df):
    """
    Filters out specific districts and adds a combined district.
    """
    # Remove "כל הארץ" and empty districts
    filtered_df = df[~df["PoliceDistrict"].isin(["כל הארץ", ""])]

    # Create "כל המחוזות" as a combined district
    all_districts = filtered_df.groupby(["Category", "Period"]).agg({"Count": "sum"}).reset_index()
    all_districts["PoliceDistrict"] = "כל המחוזות"

    # Combine the original filtered data with the combined district
    combined_df = pd.concat([filtered_df, all_districts], ignore_index=True)

    return combined_df

# Load and process data
df = load_data()
df["Quarter"] = df["Quarter"].str.extract(r"(\d)").astype(float)
df["Period"] = "לפני ה7.10"
df.loc[(df["Year"] == 2023) & (df["Quarter"] > 3), "Period"] = "אחרי ה7.10"
df.loc[df["Year"] == 2024, "Period"] = "אחרי ה7.10"

# Categorize statistic groups
df["Category"] = df["StatisticGroup"].apply(categorize_statistic_group)

# Drop rows where Category is None (uncategorized values)
df = df.dropna(subset=["Category"])

# Group by necessary fields
grouped = df.groupby(["Category", "Period", "PoliceDistrict"]).size().reset_index(name="Count")

# Apply preprocessing
grouped = preprocess_data(grouped)

# Normalize by quarter count
quarters_before = (2023 - 2020) * 4 + 3  # First quarter of 2020 to third quarter of 2023
quarters_after = 5  # Fourth quarter of 2023 to fourth quarter of 2024

grouped["NormalizedCount"] = grouped.apply(
    lambda row: row["Count"] / quarters_before if row["Period"] == "לפני ה7.10" else row["Count"] / quarters_after,
    axis=1
)

# Initialize Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("השוואת פשיעה לפי סוגי עבירות ומחוזות", style={"text-align": "center"}),

    # Dropdown for selecting district
    dcc.Dropdown(
        id="district-filter",
        # options=[
        #     {"label": district, "value": district}
        #     for district in grouped["PoliceDistrict"].unique()
        # ],
        options=[
          {"label": district, "value": district}
          for district in sorted(
              grouped["PoliceDistrict"].unique(),
              key=lambda x: (x != "כל המחוזות", x)  # Sort with "כל המחוזות" first
          )
      ],
        value="כל המחוזות",  # Default value
        clearable=False,
        style={"width": "50%"}
    ),

    dcc.Graph(id="comparison-bar-chart")
])

@app.callback(
    Output("comparison-bar-chart", "figure"),
    [Input("district-filter", "value")]
)
def update_graph(selected_district):
    filtered_df = grouped[grouped["PoliceDistrict"] == selected_district]
    pivot_df = filtered_df.pivot(index="Category", columns="Period", values="NormalizedCount").fillna(0).reset_index()

    return px.bar(
        pivot_df,
        x="Category",
        y=["לפני ה7.10", "אחרי ה7.10"],
        barmode="group",
        labels={"value": "כמות עבירות מנורמלת", "variable": "תקופה"},
        title=f"פשיעה במחוז: {selected_district}"
    ).update_layout(
        xaxis_title="סוגי עבירות",
        yaxis_title="כמות עבירות מנורמלת",
        legend_title="תקופה",
        plot_bgcolor="#f9f9f9",
        yaxis=dict(
            tickmode="linear",
            tick0=0,
            dtick=500  # Set tick interval to 500
        )
    )

if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [5]:
# import pandas as pd
# import plotly.express as px
# from dash import Dash, dcc, html, Input, Output
# import requests

# # Function to load and process data
# def load_data():
#     urls = {
#         "2020": "https://data.gov.il/api/3/action/datastore_search?resource_id=520597e3-6003-4247-9634-0ae85434b971",
#         "2021": "https://data.gov.il/api/3/action/datastore_search?resource_id=3f71fd16-25b8-4cfe-8661-e6199db3eb12",
#         "2022": "https://data.gov.il/api/3/action/datastore_search?resource_id=a59f3e9e-a7fe-4375-97d0-76cea68382c1",
#         "2023": "https://data.gov.il/api/3/action/datastore_search?resource_id=32aacfc9-3524-4fba-a282-3af052380244",
#         "2024": "https://data.gov.il/api/3/action/datastore_search?resource_id=5fc13c50-b6f3-4712-b831-a75e0f91a17e",
#     }
#     data_frames = []
#     for year, url in urls.items():
#         response = requests.get(url)
#         data = response.json()
#         records = data['result']['records']
#         df = pd.DataFrame(records)
#         df['Year'] = int(year)  # Add year column
#         df = df.dropna(subset=["StatisticGroup"])  # Remove rows with missing StatisticGroup
#         data_frames.append(df)
#     return pd.concat(data_frames, ignore_index=True)

# # Map StatisticGroup into broader categories
# def categorize_statistic_group(stat_group):
#     categories = {
#         "עבירות פליליות כלליות": ['עבירות כלפי הרכוש', 'עבירות נגד גוף', 'עבירות מין'],
#         "עבירות מוסר": ['עבירות מוסר', 'עבירות סדר ציבורי'],
#         "עבירות ביטחון": ['עבירות בטחון'],
#         "עבירות כלכליות": ['עבירות כלכליות', 'עבירות מנהליות', 'עבירות רשוי'],
#         "עבירות תנועה": ['עבירות תנועה'],
#         "עבירות מרמה": ['עבירות מרמה'],
#     }
#     for category, types in categories.items():
#         if stat_group in types:
#             return category
#     return None  # Return None for uncategorized values

# # Preprocessing function
# def preprocess_data(df):
#     """
#     Filters out specific districts and adds a combined district.
#     """
#     # Remove "כל הארץ" and empty districts
#     filtered_df = df[~df["PoliceDistrict"].isin(["כל הארץ", ""])]

#     # Create "כל המחוזות" as a combined district
#     all_districts = filtered_df.groupby(["Category", "Period"]).agg({"Count": "sum"}).reset_index()
#     all_districts["PoliceDistrict"] = "כל המחוזות"

#     # Combine the original filtered data with the combined district
#     combined_df = pd.concat([filtered_df, all_districts], ignore_index=True)

#     return combined_df

# # Load and process data
# df = load_data()
# df["Quarter"] = df["Quarter"].str.extract(r"(\d)").astype(float)
# df["Period"] = "לפני ה7.10"
# df.loc[(df["Year"] == 2023) & (df["Quarter"] > 3), "Period"] = "אחרי ה7.10"
# df.loc[df["Year"] == 2024, "Period"] = "אחרי ה7.10"

# # Categorize statistic groups
# df["Category"] = df["StatisticGroup"].apply(categorize_statistic_group)

# # Drop rows where Category is None (uncategorized values)
# df = df.dropna(subset=["Category"])

# # Group by necessary fields
# grouped = df.groupby(["Category", "Period", "PoliceDistrict"]).size().reset_index(name="Count")

# # Apply preprocessing
# grouped = preprocess_data(grouped)

# # Normalize by quarter count
# quarters_before = (2023 - 2020) * 4 + 3  # First quarter of 2020 to third quarter of 2023
# quarters_after = 5  # Fourth quarter of 2023 to fourth quarter of 2024

# grouped["NormalizedCount"] = grouped.apply(
#     lambda row: row["Count"] / quarters_before if row["Period"] == "לפני ה7.10" else row["Count"] / quarters_after,
#     axis=1
# )

# # Initialize Dash app
# app = Dash(__name__)

# app.layout = html.Div([
#     html.H1("השוואת פשיעה לפי סוגי עבירות ומחוזות", style={"text-align": "center"}),

#     # Dropdown for selecting district
#     dcc.Dropdown(
#         id="district-filter",
#         options=[
#             {"label": district, "value": district}
#             for district in sorted(
#                 grouped["PoliceDistrict"].unique(),
#                 key=lambda x: (x != "כל המחוזות", x)  # Sort with "כל המחוזות" first
#             )
#         ],
#         value="כל המחוזות",  # Default value
#         clearable=False,
#         style={"width": "50%"}
#     ),

#     dcc.Graph(id="comparison-bar-chart")
# ])

# @app.callback(
#     Output("comparison-bar-chart", "figure"),
#     [Input("district-filter", "value")]
# )
# def update_graph(selected_district):
#     filtered_df = grouped[grouped["PoliceDistrict"] == selected_district]
#     pivot_df = filtered_df.pivot(index="Category", columns="Period", values="NormalizedCount").fillna(0).reset_index()

#     return px.bar(
#         pivot_df,
#         x="Category",
#         y=["לפני ה7.10", "אחרי ה7.10"],
#         barmode="group",
#         labels={"value": "כמות עבירות מנורמלת", "variable": "תקופה"},
#         title=f"פשיעה במחוז: {selected_district}"
#     ).update_layout(
#         xaxis_title="סוגי עבירות",
#         yaxis_title="כמות עבירות מנורמלת",
#         legend_title="תקופה",
#         plot_bgcolor="#f9f9f9",
#         yaxis=dict(
#             tickmode="linear",
#             tick0=0,
#             dtick=500  # Set tick interval to 500
#         )
#     )

# if __name__ == "__main__":
#     app.run_server(debug=True)


<IPython.core.display.Javascript object>